Рудинская Елизавета, группа 6132-010402D

Лабораторная № 1

#### Задание на лабораторную работу 
##### Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
1. Найти велосипед с максимальным временем пробега.
2. Найти наибольшее геодезическое расстояние между станциями.
3. Найти путь велосипеда с максимальным временем пробега через станции.
4. Найти количество велосипедов в системе.
5. Найти пользователей потративших на поездки более 3 часов.

In [1]:
#pip install pyspark
# pip install numpy
#! pip install pandas

In [5]:
from pyspark import SparkContext, SparkConf

from typing import NamedTuple
from datetime import datetime

import pandas as pd
import os
import numpy as np

In [3]:
# настройка конфигурации
conf = SparkConf().setAppName("L1_interactive_bike_analysis").setMaster('yarn')

In [4]:
sc = SparkContext(conf=conf)

In [6]:
sc

<SparkContext master=yarn appName=L1_interactive_bike_analysis>

In [7]:
sc.parallelize([1,2,3,4,5,6]).map(lambda x: dict(os.environ)['PYTHONPATH']).first()

'/opt/mapr/spark/spark-3.1.2//python/lib/pyspark.zip:/opt/mapr/spark/spark-3.1.2//python/lib/py4j-0.10.9-src.zip:/tmp/hadoop-mapr/nm-local-dir/usercache/julia_pchelkina/filecache/19/__spark_libs__2835295945260599284.zip/spark-core_2.12-3.1.2.0-eep-800.jar:/home/julia_pchelkina/.local/lib/python3.9/site-packages:/tmp/hadoop-mapr/nm-local-dir/usercache/julia_pchelkina/appcache/application_1637674522791_0062/container_e08_1637674522791_0062_01_000002/pyspark.zip:/tmp/hadoop-mapr/nm-local-dir/usercache/julia_pchelkina/appcache/application_1637674522791_0062/container_e08_1637674522791_0062_01_000002/py4j-0.10.9-src.zip'

In [8]:
sc.parallelize([1,2,3,4,5,6]).map(lambda x: (os.environ['NM_HOST'], [12, 3])).first()

('n3.maapr.xyz', [12, 3])

In [9]:
!which python

/opt/miniconda3/bin//python


#### Загрузка и предобработка данных

In [10]:
tripData = sc.textFile("trips.csv")
stationData = sc.textFile("stations.csv")

In [40]:
# определяем первую строку в данных
tripsHeader = tripData.first()
stationsHeader = stationData.first()

# фильтруем данные на основе заданного условия (убираем заголовки (первую строку)
trips = tripData.filter(lambda row: row != tripsHeader)
stations = stationData.filter(lambda row: row != stationsHeader)

# разбиваем строки на "слова", в качестве разделителя выступает ","
trips_data = trips.map(lambda x: x.split(','))
stations_data = stations.map(lambda x: x.split(','))

# значения первой строки - заголовки
trip_list = list(enumerate(tripsHeader.split(",")))
station_list = list(enumerate(stationsHeader.split(",")))


In [42]:
# выводим первые 5 строк данных без заголовка
trips.take(5)

['4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060',
 '4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103']

In [43]:
stations.take(5)

['2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013',
 '6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013']

In [44]:
# выводим первые 2 строки данных, разбитые на "слова"
trips_data.take(2)  

[['4576',
  '63',
  '8/29/2013 14:13',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '70',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [45]:
stations_data.take(2) 

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013']]

In [46]:
# выводим значения первой строки - заголовки
trip_list

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [47]:
station_list

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

In [48]:
# создаем модель данных для Trips
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        try:
            yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            ) 
        except:
            pass

In [49]:
# создаем модель данных для Stations
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

In [50]:
# применяем модель к нашим данным
tripsInternal = trips_data.mapPartitions(initTrip)
stationsInternal = stations_data.mapPartitions(initStation)

In [51]:
# выводим первые n строк
tripsInternal.take(2)

[Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station_name='South Van Ness at Market', end_station_id='66', bike_id=520, subscription_type='Subscriber', zip_code='94127'),
 Trip(trip_id=4607, duration=70, start_date=datetime.datetime(2013, 8, 29, 14, 42), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 14, 43), end_station_name='San Jose City Hall', end_station_id='10', bike_id=661, subscription_type='Subscriber', zip_code='95138')]

In [52]:
stationsInternal.take(2)

[Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0))]

In [53]:
tripsInternal.first()

Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station_name='South Van Ness at Market', end_station_id='66', bike_id=520, subscription_type='Subscriber', zip_code='94127')

In [54]:
stationsInternal.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

#### Поиск велосипеда с максимальным временем пробега

нам нужна позиция "duration" - продолжительность

In [55]:
# список пар ключ (номер id) - значение (данные в строке с этим id)
tripsIndexed = trips_data.keyBy(lambda trip: trip[0])
tripsIndexed.take(2)

[('4576',
  ['4576',
   '63',
   '8/29/2013 14:13',
   'South Van Ness at Market',
   '66',
   '8/29/2013 14:14',
   'South Van Ness at Market',
   '66',
   '520',
   'Subscriber',
   '94127']),
 ('4607',
  ['4607',
   '70',
   '8/29/2013 14:42',
   'San Jose City Hall',
   '10',
   '8/29/2013 14:43',
   'San Jose City Hall',
   '10',
   '661',
   'Subscriber',
   '95138'])]

In [57]:
# по id велосипеда
trip_num = tripsInternal.keyBy(lambda x: x.bike_id)
trip_num.first()

(520,
 Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station_name='South Van Ness at Market', end_station_id='66', bike_id=520, subscription_type='Subscriber', zip_code='94127'))

In [58]:
# группируем данные по выбранному столбцу и выполняем различные операции
group_trip_num = trip_num\
                 .mapValues(lambda x: x.duration/60/60)\
                 .reduceByKey(lambda d1, d2: d1 + d2)


In [60]:
%%time

# находим наибольшее расстояние, замеряем время обработки
max_duration = group_trip_num.reduce(lambda d1, d2: d1 if d1[1] > d2[1] else d2)


CPU times: user 10.5 ms, sys: 1.36 ms, total: 11.9 ms
Wall time: 209 ms


In [29]:
# max_duration

In [61]:
max_time = int(max_duration[1]*60*60)

print('Maximum duration has bike with id = ', max_duration[0])


Maximum duration has bike with id =  535


In [62]:
# Делаем то же самое в Pandas для проверки
df = pd.read_csv('trips.csv')
m1 = str(df['duration'].max())
# Находим еще одно наибольшее значение
df_new = df.loc[df['duration'] != int(m1)]
m2 = df_new['duration'].max()

print('maximum duration:', m1, m2)

maximum duration: 17270400 2137000


In [63]:
# выводим данные по двум самым длительным поездкам:
df_max_duration = df.query('duration == 17270400 or duration == 2137000')
df_max_duration.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
382718,825850,2137000,6/28/2015 21:50,Market at Sansome,77,7/23/2015 15:27,Yerba Buena Center of the Arts (3rd @ Howard),68,466,Customer,97213
573566,568474,17270400,12/6/2014 21:59,South Van Ness at Market,66,6/24/2015 20:18,2nd at Folsom,62,535,Customer,95531


Вывод: результаты совпали. время обработки в первом случае меньше. Наибольший пробег имеет велосипед с номером id = 535

### Поиск наибольшего геодезического расстояния между станциями

Евклидова метрика не подходит, т.к. предназначена для вычисления расстояния на плоскости, поэтому будем использовать формулу для поиска расстояния между точками на сфере.

Будем использовать версинус и гаверсинус:

versin(φ) = 2 * (sin(φ/2))^2

hav(φ) = versin(φ) / 2 = (sin(φ/2))^2

Гаверсинус центрального угла между любыми двумя точками на сфере вычисляется по формуле:

hav(d/r) = hav(φ2 - φ1) + cos(φ1)*cos(φ2)*hav(λ2 - λ1)
- d - это центральный угол между двумя точками, лежащими на большом круге
- r - радиус сферы
- φ1, φ2 - широта первой и второй точек в радианах
- λ1, λ2 - долгота первой и второй точек в радианах

Тогда 

d = 2r*arcsin(sqrt(hav(φ2 - φ1) + cos(φ1)*cos(φ2)*hav(λ2 - λ1))), где

hav(φ2 - φ1) = (sin((φ2 - φ1)/2))^2

hav(λ2 - λ1) = (sin((λ2 - λ1)/2))^2


Средний радиус Земли R = 6371210 м или 6371 км

Экваториальный радиус Земли RЭ = 6378,245 м.

Полярный радиус Земли RП = 6356,830 м.

Длина дуги меридиана (дуги экватора, дуги окружности большого круга) в 1°, 1′ и 1″ равна соответственно: 111 197 м (111,2 км), 1852 м (1,852 км) и 30,9 м.


In [78]:
from math import pi, radians, cos, sin, asin, sqrt, atan, degrees
from enum import Enum

# средний радиус земли
Rad_mean = 6371210

In [85]:
# получаем 3 кортежа: 1 - названия двух станций, 2 и 3 - данные по этим станциям
distance = stationsInternal.cartesian(stationsInternal)\
                         .filter(lambda x: x[0].station_id != x[1].station_id)\
                         .keyBy(lambda x: (x[0].name, x[1].name))

In [86]:
distance.take(2)

[(('San Jose Diridon Caltrain Station', 'San Jose Civic Center'),
  (Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
   Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0)))),
 (('San Jose Diridon Caltrain Station', 'Santa Clara at Almaden'),
  (Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
   Station(station_id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dockcount=11, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))))]

In [87]:
def geodetic_distance(lat1, long1, lat2, long2):
        
    # перевод в радианы
    lat_1 = radians(lat1)
    long_1 = radians(long1)
    lat_2 = radians(lat2)
    long_2 = radians(long2)
    
    # разность между широтой и долготой
    lat_1_2 = lat_2 - lat_1
    long_1_2 = long_2 - long_1
    
    # гаверсинусы
    hav_lat = (sin(lat_1_2/2))**2
    hav_long = (sin(long_1_2/2))**2
    
    # получаем расстояние по формуле
    d = 2*Rad_mean*asin(sqrt(hav_lat + cos(lat_1)*cos(lat_2)*hav_long))
    
    return d

In [88]:
# к данным о каждой паре станций дописываем расстояние между ними
distance_pair = distance.mapValues(lambda x: [x[0], x[1], geodetic_distance(x[0].lat, x[0].long, x[1].lat, x[1].long)])                                        

In [89]:
distance_pair.take(3)

[(('San Jose Diridon Caltrain Station', 'San Jose Civic Center'),
  [Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
   Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0)),
   1137.125987953115]),
 (('San Jose Diridon Caltrain Station', 'Santa Clara at Almaden'),
  [Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
   Station(station_id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dockcount=11, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
   770.7299940340974]),
 (('San Jose Diridon Caltrain Station', 'Adobe on Almaden'),
  [Station(station_id

In [90]:
%%time

# находим максимальный элемент среди всех расстояний , замеряем время обработки
max_distance = distance_pair.reduce(lambda d1, d2: d1 if d1[1] > d2[1] else d2)


CPU times: user 10.9 ms, sys: 5.25 ms, total: 16.2 ms
Wall time: 835 ms


In [102]:
print('Maximum geodetic distance between stations: ', max_distance[0])
print('distance = ', max_distance[1][2])

Maximum geodetic distance between stations:  ('Ryland Park', 'Mezes Park')
distance =  34319.03961961622


#### Поиск пути велосипеда с максимальным временем пробега через станции.

In [112]:
trip_list

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [105]:
# уже найденный bike_id 
print('Maximum duration has bike with id = ', max_duration[0])

Maximum duration has bike with id =  535


In [118]:
# все данные по этому bike_id
bike_data = tripsInternal.filter(lambda x: x.bike_id == max_duration[0])
bike_data.take(3)

[Trip(trip_id=5067, duration=423, start_date=datetime.datetime(2013, 8, 29, 21, 38), start_station_name='San Francisco Caltrain (Townsend at 4th)', start_station_id=70, end_date=datetime.datetime(2013, 8, 29, 21, 45), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=535, subscription_type='Subscriber', zip_code='94133'),
 Trip(trip_id=4966, duration=1245, start_date=datetime.datetime(2013, 8, 29, 19, 32), start_station_name='Post at Kearney', start_station_id=47, end_date=datetime.datetime(2013, 8, 29, 19, 53), end_station_name='San Francisco Caltrain (Townsend at 4th)', end_station_id='70', bike_id=535, subscription_type='Customer', zip_code='94123'),
 Trip(trip_id=5199, duration=498, start_date=datetime.datetime(2013, 8, 30, 9, 10), start_station_name='Market at Sansome', start_station_id=77, end_date=datetime.datetime(2013, 8, 30, 9, 19), end_station_name='2nd at South Park', end_station_id='64', bike_id=535, subscription_type='Customer', zip_

In [119]:
# сортируем 
bike_data_sort = bike_data.sortBy(lambda x: x.start_date)
bike_data_sort.take(3)

[Trip(trip_id=4966, duration=1245, start_date=datetime.datetime(2013, 8, 29, 19, 32), start_station_name='Post at Kearney', start_station_id=47, end_date=datetime.datetime(2013, 8, 29, 19, 53), end_station_name='San Francisco Caltrain (Townsend at 4th)', end_station_id='70', bike_id=535, subscription_type='Customer', zip_code='94123'),
 Trip(trip_id=5067, duration=423, start_date=datetime.datetime(2013, 8, 29, 21, 38), start_station_name='San Francisco Caltrain (Townsend at 4th)', start_station_id=70, end_date=datetime.datetime(2013, 8, 29, 21, 45), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=535, subscription_type='Subscriber', zip_code='94133'),
 Trip(trip_id=5179, duration=842, start_date=datetime.datetime(2013, 8, 30, 8, 40), start_station_name='San Francisco Caltrain 2 (330 Townsend)', start_station_id=69, end_date=datetime.datetime(2013, 8, 30, 8, 54), end_station_name='Market at Sansome', end_station_id='77', bike_id=535, subscription

In [134]:
# путь (промежутки)
bike_way = bike_data_sort.map(lambda x: (x.start_station_name, x.end_station_name))
#bike_way1.saveAsTextFile('bike_way.txt')
bike_way.take(10)

[('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)'),
 ('San Francisco Caltrain (Townsend at 4th)',
  'San Francisco Caltrain 2 (330 Townsend)'),
 ('San Francisco Caltrain 2 (330 Townsend)', 'Market at Sansome'),
 ('Market at Sansome', '2nd at South Park'),
 ('2nd at Townsend', 'Davis at Jackson'),
 ('San Francisco City Hall', 'Civic Center BART (7th at Market)'),
 ('Civic Center BART (7th at Market)', 'Post at Kearney'),
 ('Post at Kearney', 'Embarcadero at Sansome'),
 ('Embarcadero at Sansome', 'Washington at Kearney'),
 ('Washington at Kearney', 'Market at Sansome')]

In [135]:
# станции поочередности
bike_way1 = bike_way.map(lambda x: x[0])
#bike_way1.saveAsTextFile('bike_way1.txt')
bike_way1.take(10)

['Post at Kearney',
 'San Francisco Caltrain (Townsend at 4th)',
 'San Francisco Caltrain 2 (330 Townsend)',
 'Market at Sansome',
 '2nd at Townsend',
 'San Francisco City Hall',
 'Civic Center BART (7th at Market)',
 'Post at Kearney',
 'Embarcadero at Sansome',
 'Washington at Kearney']

#### Поиск количества велосипедов в системе.

In [136]:
trip_list

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [137]:
station_list

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

Поле "bike_id" встречается только в trips.
Находим количество неповторяющихся "bike_id"

In [152]:
# Сортируем по id велосипеда
bikes_num = tripsInternal.map(lambda x: x.bike_id)
bikes_num.take(10)

[520, 661, 48, 26, 319, 527, 679, 687, 553, 107]

In [154]:
# отобрали уникальные
bike_count = bikes_num.distinct().collect()
print('Kоличество велосипедов в системе = ', len(bike_count))

Kоличество велосипедов в системе =  700


#### 5. Найти пользователей потративших на поездки более 3 часов.

In [166]:
%%time

# фильтруем по условию, считаем количество, замеряем время обработки
long_trip_duration = tripsInternal.filter(lambda x: x.duration > 3*60*60).map(lambda y: y.zip_code)
trip_count = long_trip_duration.count()

CPU times: user 7.64 ms, sys: 10.3 ms, total: 18 ms
Wall time: 14.1 s


In [167]:
print('Kоличество пользователей потративших на поездки более 3 часов = ', trip_count)

Kоличество пользователей потративших на поездки более 3 часов =  8322
